In [11]:
import lmfit

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import warnings
warnings.filterwarnings('ignore')
import numpy as np

import sys
import os
import shutil
cwd = os.getcwd()
python_dir = cwd.split('/')[0:-3]
python_dir.append('Python Library')
s='/'
python_dir = s.join(python_dir)
sys.path.insert(0,python_dir)
sys.path.insert(0,'/opt/Mantid/bin/')
from lmfit import Model
import scipy
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import import_material_cif
from import_material_cif import import_NIST_table
from import_material_cif import Material
from mantid.simpleapi import *
from THfuncs import IEXY_data
import THfuncs
import MDUtils as mdu
import matplotlib
import matplotlib.ticker as ticker
import glob
import re
import lmfit

from ipywidgets import *
%matplotlib notebook
import ppms_class
from ppms_class import HCdata
from import_material_cif import Material
import matplotlib.pyplot as plt
from PyNuclearSchottky import NuclearSchottky

cifname = 'baco2(as2o4)2.cif'
nist_file = python_dir+'/nist_scattering_table.txt'

BCAO = Material(cifname,suppress_print=True,nist_data=nist_file)

def get_mesh_from_dat(dat):
    q_unique = np.unique(dat[:,2])
    e_unique = np.unique(dat[:,3])
    Q,E = np.meshgrid(q_unique,e_unique)
    I = np.zeros(np.shape(Q))
    for i in range(len(dat)):
        intensity = dat[i,0]
        err = dat[i,1]
        q= dat[i,2]
        e = dat[i,3]
        q_i = np.argmin(np.abs(q_unique-q))
        e_i = np.argmin(np.abs(e_unique-e))
        I[e_i,q_i]=intensity
    return Q,E,I

 Material.formula_weight=(val)


In [15]:
#Plot the 3D Brillioun zone
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt

#High symmetry points in the hexagonal BZ
Mpt = np.array([0.5,0,0])
Kpt = np.array([1.0/3.0,1.0/3.0,0])
Hpt = np.array([1.0/3.0,1.0/3.0,0.5])
hvec = BCAO.u
kvec = BCAO.v
lvec = BCAO.w

#Generate vertices
def gen_Kpts(hvec,kvec,lvec):
    #returns a list of the K pts at the edge of the BZ
    K1 = np.array(hvec/3 + kvec/3)
    Kpts = [K1]
    #Rotate by 60 deg five times
    prevK=K1
    R = np.array([[np.cos(np.pi/3),-np.sin(np.pi/3),0],[np.sin(np.pi/3),np.cos(np.pi/3),0],[0,0,1]])
    for i in [1,2,3,4,5]:
        newK = np.matmul(R,prevK)
        Kpts.append(newK)
        prevK=newK
    #shift by pm 1/2 L
    outK = []
    for pt in Kpts:
        outK.append(pt+lvec/2.0)
        outK.append(pt-lvec/2.0)
    return np.array(outK)
def gen_Mpts(hvec,kvec,lvec):
    #returns a list of the K pts at the edge of the BZ
    M1 = np.array(hvec/2)
    Mpts = [M1]
    #Rotate by 60 deg  around L vector five times
    prevM=M1
    R = np.array([[np.cos(np.pi/3),-np.sin(np.pi/3),0],[np.sin(np.pi/3),np.cos(np.pi/3),0],[0,0,1]])
    for i in [1,2,3,4,5]:
        newM = np.matmul(R,prevM)
        Mpts.append(newM)
        prevM=newM
    #shift by pm 1/2 L
    outM = []
    for pt in Mpts:
        outM.append(pt+lvec/2.0)
        outM.append(pt-lvec/2.0)
    return np.array(outM)

def gen_BZ(hvec,kvec,lvec):
    #Returns the edges of the BZ
    K1 = np.array(hvec/3 + kvec/3)
    Kpts = [K1]
    #Rotate by 60 deg around L vector five times
    prevK=K1
    R = np.array([[np.cos(np.pi/3),-np.sin(np.pi/3),0],[np.sin(np.pi/3),np.cos(np.pi/3),0],[0,0,1]])
    for i in [1,2,3,4,5]:
        newK = np.matmul(R,prevK)
        Kpts.append(newK)
        prevK=newK
    #shift by pm 1/2 L
    outK=[]
    topK = []
    bottomK = []
    vert_edges = []
    for pt in Kpts:
        outK.append(pt+lvec/2.0)
        outK.append(pt-lvec/2.0)
        vert_edges.append([pt+lvec/2.0,pt-lvec/2.0])
        topK.append(pt+lvec/2.0)
        bottomK.append(pt-lvec/2.0)
    for i,pt in enumerate(topK):
        pt0 =  topK[i-1]
        vert_edges.append([pt0,pt])
    for i,pt in enumerate(bottomK):
        pt0=bottomK[i-1]
        vert_edges.append([bottomK[i-1],pt])
    return np.array(outK),vert_edges


fig,ax = plt.subplots(1,2,figsize=(8,4),subplot_kw={'projection':'3d'},sharex=True,sharey=True)
ax[0].set_xlabel('$k_x$')
ax[0].set_ylabel('$k_y$')
ax[0].set_zlabel('$k_z$')
ax[1].set_xlabel('$k_x$')
ax[1].set_ylabel('$k_y$')
ax[1].set_zlabel('$k_z$')
ax[1].view_init(elev = 90, azim = -90)

#Plot reciprocal lattice vectors

for ax_plt in [ax[0],ax[1]]:
    ax_plt.quiver3D(0,0,0.0,hvec[0],hvec[1],hvec[2],color='k',pivot='tail',\
                 arrow_length_ratio = 0.2)
    ax_plt.quiver3D(0,0,0.0,kvec[0],kvec[1],kvec[2],color='k',pivot='tail',\
                 arrow_length_ratio=0.2)
    ax_plt.quiver3D(0,0,0.0,lvec[0],lvec[1],lvec[2],color='k',pivot='tail',\
                 arrow_length_ratio=0.2)

#Plot vertices 
Kpts,BZ_edges = gen_BZ(hvec,kvec,lvec)
Mpts = gen_Mpts(hvec,kvec,lvec)
for ax_plt in [ax[0],ax[1]]:
    ax_plt.plot(Kpts[:,0],Kpts[:,1],Kpts[:,2],color='b',marker='o',ls=' ',ms=3,zorder=10)
    ax_plt.plot(Mpts[:,0],Mpts[:,1],Mpts[:,2],color='k',marker='o',ls=' ',ms=3,zorder=11)

    for edge in BZ_edges:
        pt0 = np.array(edge[0])
        pt1=np.array(edge[1])
        X = np.array([pt0[0],pt1[0]])
        Y = np.array([pt0[1],pt1[1]])
        Z = np.array([pt0[2],pt1[2]])
        ax_plt.plot(X,Y,Z,\
                    marker=' ',ls='-',color='c',alpha=0.5,zorder=5)      
xmax = 0.25
zmax = xmax*BCAO.astar/BCAO.cstar
ax[0].set_xlim(-xmax,xmax)
ax[0].set_ylim(-xmax,xmax)
ax[0].set_zlim(-xmax,xmax)
ax[0].grid(False)
ax[1].grid(False)

#Plot the BZ Path for spag plot
path = np.array([[0.5,0,0],[1.0/3.0,1.0/3.0,0.0],[0.0,1.0,0.0],[.0,0.5,0.0],[1.0/3.0,1.0/3.0,0.0],\
                [0.5,0.5,0.0],[1.0,0.0,0.0],[0.5,0.0,0.0]])

Qpath = path[:,0]*hvec[:,np.newaxis]+path[:,1]*kvec[:,np.newaxis]\
        +path[:,2]*lvec[:,np.newaxis]
Qpath=Qpath.T
ax[0].plot(Qpath[:,0],Qpath[:,1],Qpath[:,2],marker='*',color='g',ls='-',zorder=20)
ax[1].plot(Qpath[:,0],Qpath[:,1],Qpath[:,2],marker='*',color='g',ls='-',zorder=20)

#Plot the rotated coordinate frame after field reorientation, using two rotations. 
#First rotation is about the x-axis, second about y-axis and random.
unit_K = np.array([1.0,0.0,0.0])
theta = -np.pi/2 + 15.0*np.pi/180.0
Kmat = np.array([[0.0,-unit_K[2],unit_K[1]],[unit_K[2],0,-unit_K[0]],[-unit_K[1],unit_K[0],0]])
Rmat = np.eye(3)+np.sin(theta)*Kmat + (1.0-np.cos(theta))*np.matmul(Kmat,Kmat)

#Second rotation is about the y-axis
unit_K = np.array([0.0,0.0,1.0])
delphi = 0.0
phi = (delphi+90.0)*np.pi/180.0
Kmat = np.array([[0.0,-unit_K[2],unit_K[1]],[unit_K[2],0,-unit_K[0]],[-unit_K[1],unit_K[0],0]])
Rmat2 = np.eye(3)+np.sin(phi)*Kmat + (1.0-np.cos(phi))*np.matmul(Kmat,Kmat)

#Optional final rotation about the z-axis\
unit_K = np.array([0.0,1.0,0.0])
eta = 0.0*np.pi/180.0
Kmat = np.array([[0.0,-unit_K[2],unit_K[1]],[unit_K[2],0,-unit_K[0]],[-unit_K[1],unit_K[0],0]])
Rmat3 = np.eye(3)+np.sin(eta)*Kmat + (1.0-np.cos(eta))*np.matmul(Kmat,Kmat)

Rmat = np.matmul(Rmat,Rmat2)
Rmat = np.matmul(Rmat,Rmat3)
#Repeat the same proecedure as before, rotating every point before plotting


for ax_plt in [ax[0],ax[1]]:
    for i in range(len(Kpts[:,0])):
        pt = np.array(Kpts[i])
        rotpt = pt*np.cos(theta) + np.cross(unit_K,pt)+unit_K*np.dot(unit_K,pt)*(1.0-np.cos(theta))

        #ax_plt.plot(rotpt[0],rotpt[1],rotpt[2],mfc='r',color='r',marker='s',ls=' ',ms=3,zorder=10)
    for i in range(len(Mpts[:,0])):
        pt = Mpts[i]
        rotpt = np.matmul(Rmat,pt)
        ax_plt.plot(rotpt[0],rotpt[1],rotpt[2],color='m',mfc='m',marker='s',ls=' ',ms=3,zorder=11)

    for edge in BZ_edges:
        pt0 = np.array(edge[0])
        pt1=np.array(edge[1])
        pt0 = np.matmul(Rmat,pt0)
        pt1 = np.matmul(Rmat,pt1)
        X = np.array([pt0[0],pt1[0]])
        Y = np.array([pt0[1],pt1[1]])
        Z = np.array([pt0[2],pt1[2]])
        ax_plt.plot(X,Y,Z,\
                    marker=' ',ls='-',color='m',alpha=0.5,zorder=5)    
#While visually nice, now project the original points onto the new frame. 
hrot = np.matmul(Rmat,hvec)
krot = np.matmul(Rmat,kvec)
lrot = np.matmul(Rmat,lvec)

for ax_plt in [ax[0],ax[1]]:
    ax_plt.quiver3D(0,0,0.0,hrot[0],hrot[1],hrot[2],color='r',pivot='tail',\
                 arrow_length_ratio = 0.2)
    ax_plt.quiver3D(0,0,0.0,krot[0],krot[1],krot[2],color='r',pivot='tail',\
                 arrow_length_ratio=0.2)
    ax_plt.quiver3D(0,0,0.0,lrot[0],lrot[1],lrot[2],color='r',pivot='tail',\
                 arrow_length_ratio=0.2)

#Now we need to express the BZ points in terms of the new basis vectors using matrix formalism
#Columns are H, K, L, rotated
B = np.array([hrot,krot,lrot]).T
Binv = np.linalg.inv(B)
Hvec_init = np.array([0,0,1.0])
Hvec_rot = np.matmul(Binv,Hvec_init)


rotQpath = np.zeros(np.shape(Qpath))
for i,pt in enumerate(Qpath):
    rotpt = np.matmul(Binv,pt)
    #Pt is now represented in the new basis
    rotQpath[i]=rotpt

rotQ_cart = rotQpath[:,0]*hrot[:,np.newaxis]+rotQpath[:,1]*krot[:,np.newaxis]+\
                rotQpath[:,2]*lrot[:,np.newaxis]
rotQ_cart=rotQ_cart.T
#Plot these points to make sure they match.
ax[0].plot(rotQ_cart[:,0],rotQ_cart[:,1],rotQ_cart[:,2],marker='^',color='k',ls='-',zorder=24)
ax[1].plot(rotQ_cart[:,0],rotQ_cart[:,1],rotQ_cart[:,2],marker='^',color='r',ls='--',alpha=0.8,zorder=25)
fig.show()
#Finally, turn the rotated frame coordinates into HKL indices for spinW input. 
rotQ_cart_HKL = rotQpath
#Save as a txt file:
#np.savetxt(f"Path_indices_hkl_rotated/BCAO_phi_{delphi:.2f}_theta_{theta*180.0/np.pi:.2f}.csv",\
#          rotQ_cart_HKL,delimiter=',',header='Hbar,Kbar,Lbar')

<IPython.core.display.Javascript object>

In [14]:
#Do this without the visualization and save to a number of files. 
nSample=50
import random
phi1_center = 130.0
phi2_center = 35.0 #Controls field angle in ab plane
deltatheta = 12.0 #Rotation out of plane
eta_center = 0.0 #Rotates final sampe configuration relative to z-axis
phi_list1 = [random.uniform(129,1) for _ in range(nSample)] #Degrees
phi_list2 = [random.uniform(phi2_center-5.0,phi2_center+5.0) for _ in range(nSample)] #Degrees
eta_list = [random.uniform(eta_center-5.0,eta_center+5.0) for _ in range(nSample)] #degrees
phi_list = np.array(phi_list2)
#phi_list = np.array(phi_list1+phi_list2)
#phi_list = np.array([145.0])
#Delete the current indices in the folder
folder = os.getcwd()+'/Spinw/Path_indices_hkl_rotated/'
files = glob.glob(folder+'*.csv')
for f in files:
    os.remove(f)
for delphi in phi_list:
    #High symmetry points in the hexagonal BZ
    Mpt = np.array([0.5,0,0])
    Kpt = np.array([1.0/3.0,1.0/3.0,0])
    Hpt = np.array([1.0/3.0,1.0/3.0,0.5])
    hvec = BCAO.u
    kvec = BCAO.v
    lvec = BCAO.w
    #Plot vertices 
    Kpts,BZ_edges = gen_BZ(hvec,kvec,lvec)
    Mpts = gen_Mpts(hvec,kvec,lvec)

    #Plot BZ Path for spag plot
    path = np.array([[0.5,0,0],[1.0/3.0,1.0/3.0,0.0],[0.0,1.0,0.0],[.0,0.5,0.0],[1.0/3.0,1.0/3.0,0.0],\
                    [0.5,0.5,0.0],[1.0,0.0,0.0],[0.5,0.0,0.0]])
    Qpath = path[:,0]*hvec[:,np.newaxis]+path[:,1]*kvec[:,np.newaxis]\
            +path[:,2]*lvec[:,np.newaxis]
    Qpath=Qpath.T

    #Plot the rotated coordinate frame after field reorientation, using two rotations. 
    #First rotation is about the x-axis, second about y-axis and random.
    unit_K = np.array([1.0,0.0,0.0])
    theta = -np.pi/2 + deltatheta*np.pi/180.0
    Kmat = np.array([[0.0,-unit_K[2],unit_K[1]],[unit_K[2],0,-unit_K[0]],[-unit_K[1],unit_K[0],0]])
    Rmat = np.eye(3)+np.sin(theta)*Kmat + (1.0-np.cos(theta))*np.matmul(Kmat,Kmat)

    #Second rotation is about the y-axis
    unit_K = np.array([0.0,1.0,0.0])
    #delphi = 0.0
    phi = (delphi+180.0)*np.pi/180.0
    Kmat = np.array([[0.0,-unit_K[2],unit_K[1]],[unit_K[2],0,-unit_K[0]],[-unit_K[1],unit_K[0],0]])
    Rmat2 = np.eye(3)+np.sin(phi)*Kmat + (1.0-np.cos(phi))*np.matmul(Kmat,Kmat)
    #Optional final rotation about the z-axis\
    unit_K = np.array([0.0,1.0,0.0])
    eta = (eta_list[i]+eta_center)*np.pi/180.0
    Kmat = np.array([[0.0,-unit_K[2],unit_K[1]],[unit_K[2],0,-unit_K[0]],[-unit_K[1],unit_K[0],0]])
    Rmat3 = np.eye(3)+np.sin(eta)*Kmat + (1.0-np.cos(eta))*np.matmul(Kmat,Kmat)

    Rmat = np.matmul(Rmat,Rmat2)
    #Rmat = np.matmul(Rmat,Rmat3)

    # project the original points onto the new frame. 
    hrot = np.matmul(Rmat,hvec)
    krot = np.matmul(Rmat,kvec)
    lrot = np.matmul(Rmat,lvec)

    #Now we need to express the BZ points in terms of the new basis vectors using matrix formalism
    #Columns are H, K, L, rotated
    B = np.array([hrot,krot,lrot]).T
    Binv = np.linalg.inv(B)
    Hvec_init = np.array([0,0,1.0])
    Hvec_rot = np.matmul(Binv,Hvec_init)

    rotQpath = np.zeros(np.shape(Qpath))
    for i,pt in enumerate(Qpath):
        rotpt = np.matmul(Binv,pt)
        #Pt is now represented in the new basis
        rotQpath[i]=rotpt

    rotQ_cart = rotQpath[:,0]*hrot[:,np.newaxis]+rotQpath[:,1]*krot[:,np.newaxis]+\
                    rotQpath[:,2]*lrot[:,np.newaxis]
    rotQ_cart=rotQ_cart.T

    #Finally, turn the rotated frame coordinates into HKL indices for spinW input. 
    rotQ_cart_HKL = rotQpath
    #Save as a txt file:
    np.savetxt(f"SpinW/Path_indices_hkl_rotated/BCAO_phi_{delphi:.2f}_theta_{theta*180.0/np.pi:.2f}.csv",\
              rotQ_cart_HKL,delimiter=',',header='Hbar,Kbar,Lbar')